In [3]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import label_binarize

In [5]:
dataset_iris = datasets.load_iris()
iris = np.column_stack([dataset_iris['data'], dataset_iris['target']])
np.random.shuffle(iris)
#print(iris)
target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])

# Parameters
learning_rate = 0.001
training_epochs = 150
batch_size = 30
display_step = 1

# Network Parameters
n_hidden_1 = 40 # 1st layer number of neurons
n_hidden_2 = 40 # 2nd layer number of neurons
n_input = 4 # target + 3 classes
n_classes = 3 #total classes

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(150/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = iris[i:i+batch_size, :4], target_multiclass[i:i+batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: iris[:,:-1], Y: target_multiclass}))

Epoch: 0001 cost=513.658752441
Epoch: 0002 cost=457.783990479
Epoch: 0003 cost=402.369616699
Epoch: 0004 cost=347.381182861
Epoch: 0005 cost=292.771179199
Epoch: 0006 cost=238.468118286
Epoch: 0007 cost=184.385116577
Epoch: 0008 cost=132.643714905
Epoch: 0009 cost=101.520230103
Epoch: 0010 cost=83.043208313
Epoch: 0011 cost=66.448638153
Epoch: 0012 cost=46.603003693
Epoch: 0013 cost=25.894636917
Epoch: 0014 cost=8.675681019
Epoch: 0015 cost=1.156802750
Epoch: 0016 cost=0.403132123
Epoch: 0017 cost=0.378287327
Epoch: 0018 cost=0.411062331
Epoch: 0019 cost=0.451287946
Epoch: 0020 cost=0.431290786
Epoch: 0021 cost=0.415159543
Epoch: 0022 cost=0.400803161
Epoch: 0023 cost=0.382755786
Epoch: 0024 cost=0.366237957
Epoch: 0025 cost=0.358089051
Epoch: 0026 cost=0.352516976
Epoch: 0027 cost=0.341335906
Epoch: 0028 cost=0.326946491
Epoch: 0029 cost=0.311892921
Epoch: 0030 cost=0.298015704
Epoch: 0031 cost=0.285834967
Epoch: 0032 cost=0.273692115
Epoch: 0033 cost=0.260264994
Epoch: 0034 cost=0.24